<a href="https://colab.research.google.com/github/deondrae4088/WebMd_chat/blob/Project-3-(Gabe's-Branch)/AlmostCompleteProject3Gabe'sBranch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install --upgrade tensorflow kagglehub

import tensorflow as tf
import pathlib
import kagglehub
import os
import zipfile
import glob
import PIL
from collections import defaultdict
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split

dataset_names = [
    "shubhamgoel27/dermnet",
    "pkdarabi/brain-tumor-image-dataset-semantic-segmentation",
    "paultimothymooney/chest-xray-pneumonia",
    "maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset",
    "aryashah2k/breast-ultrasound-images-dataset"
]

# First part of your code: Download and extract datasets
extracted_dirs = []

for name in dataset_names:
  try:
        download_path = kagglehub.dataset_download(name)
        print(f"Downloaded '{name}' to: {download_path}")
        extract_path = os.path.join(os.getcwd(), name.split('/')[1]) if download_path.endswith(".zip") else download_path
        if download_path.endswith(".zip"):
            with zipfile.ZipFile(download_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"Extracted '{name}' to: {extract_path}")
        extracted_dirs.append(pathlib.Path(extract_path))
  except Exception as e:  # Define e here as the exception variable
        print(f"Error opening image: {e}")



# Second part of your code: Improved one-liner version for extracting and organizing
extracted_dirs = [pathlib.Path(os.path.join(os.getcwd(), name.split('/')[1])) if (download_path := kagglehub.dataset_download(name)).endswith(".zip") and (os.makedirs(os.path.join(os.getcwd(), name.split('/')[1]), exist_ok=True) or True) and zipfile.ZipFile(download_path, 'r').extractall(os.path.join(os.getcwd(), name.split('/')[1])) else pathlib.Path(download_path)
                  for name in dataset_names
                  if print(f"Downloaded '{name}' to: {download_path}") or True]

# Gather all image files across datasets
all_images = []
for img_dir in extracted_dirs:
    print(f"Processing: {img_dir}")
    for ext in ['**/*.jpg', '**/*.jpeg', '**/*.png']:
        matching_images = list(img_dir.glob(ext))
        print(f"  Found {len(matching_images)} images with extension {ext}")
        all_images.extend(matching_images)

print(f"\nTotal images found across {len(extracted_dirs)} directories: {len(all_images)}")
all_images and (print(f"First image found: {all_images[0]}") or True)


Downloaded 'shubhamgoel27/dermnet' to: /root/.cache/kagglehub/datasets/shubhamgoel27/dermnet/versions/1
Downloaded 'pkdarabi/brain-tumor-image-dataset-semantic-segmentation' to: /root/.cache/kagglehub/datasets/pkdarabi/brain-tumor-image-dataset-semantic-segmentation/versions/1
Downloaded 'paultimothymooney/chest-xray-pneumonia' to: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2
Downloaded 'maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset' to: /root/.cache/kagglehub/datasets/maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset/versions/2
Downloaded 'aryashah2k/breast-ultrasound-images-dataset' to: /root/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1
Downloaded 'shubhamgoel27/dermnet' to: /root/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1
Downloaded 'pkdarabi/brain-tumor-image-dataset-semantic-segmentation' to: /root/.cache/kagglehub/datasets/shubhamgoel27/dermnet/versio

True

In [19]:
# Organize images by class (folder structure)
organize_images_by_class = lambda root_dir, image_extensions=['*.jpg', '*.jpeg', '*.png']: defaultdict(list, {
    item.name: [p for ext in image_extensions for p in item.glob(ext)]
    for item in pathlib.Path(root_dir).iterdir() if item.is_dir()
})

# Organize all images
all_organized_images = defaultdict(list)
[all_organized_images.update(organize_images_by_class(dataset_dir)) for dataset_dir in extracted_dirs]

# Print the organized image details
print("\nOrganized images by class:") if all_organized_images else print("\nNo images organized by class found.")
[print(f"Class: {cls}, Found {len(imgs)} images") for cls, imgs in all_organized_images.items()]



Organized images by class:
Class: train, Found 1502 images
Class: test, Found 215 images
Class: valid, Found 429 images
Class: chest_xray, Found 0 images
Class: masks, Found 2729 images
Class: frames, Found 2729 images
Class: Dataset_BUSI_with_GT, Found 0 images


[None, None, None, None, None, None, None]

In [ ]:
if all_organized_images:
    # Model Setup
    Sequential = tf.keras.models.Sequential
    layers = tf.keras.layers
    img_height = 224
    img_width = 224
    num_classes = len(all_organized_images)

    # Define the model
    model = Sequential([
        layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Image preprocessing
    load_and_preprocess_image = lambda img_path, target_size=(img_height, img_width): image.img_to_array(image.load_img(img_path, target_size=target_size)) / 255.0

    # Prepare dataset
    X = np.array([load_and_preprocess_image(str(img_path)) for class_name, img_paths in all_organized_images.items() for img_path in img_paths])
    y = np.array([i for i, class_name in enumerate(all_organized_images.keys()) for _ in all_organized_images[class_name]])
    y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

    # Print shapes
    print("\nShape of image data (X):", X.shape)
    print("Shape of labels (y):", y.shape)

    # Split into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

else:
    print("\nNo organized images found to build the model.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       6,422,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,447,143 (24.59 MB)

 Trainable params: 6,447,143 (24.59 MB)

 Non-trainable params: 0 (0.00 B)


Shape of image data (X): (7604, 224, 224, 3)
Shape of labels (y): (7604, 7)


In [1]:
# This is a simple Gradio app that uses a *trained* model to classify images
# based on the labels it learned during training.
!pip install gradio
!pip install gradio torch torchvision pillow
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image



# Assume you have the number of classes from your training data
num_classes = 7  # Replace with the actual number of classes

# Load your trained model
model = model(num_classes)

try:
    model.load_state_dict(torch.load('path/to/your/trained_model.pth'))
    model.eval()  # Set the model to evaluation mode
    print("Trained model loaded successfully.")
except FileNotFoundError:
    print("Error: Trained model file not found. Please provide the correct path.")
    exit()

# Assume you have the class labels corresponding to the model's output
# Replace this with your actual list of class labels
class_labels = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4',
                'class_5', 'class_6', 'class_7']

# Define the transformations used during training
# Make sure these are the *same* transformations you applied to your training data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Or the size you trained on
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Or your training normalization
])

def classify_image(img):
    """Classifies the input image using the loaded trained model."""
    try:
        img = Image.fromarray(img).convert('RGB') # Ensure image is RGB
        img_t = transform(img).unsqueeze(0)

        with torch.no_grad():
            out = model(img_t)
            probabilities = F.softmax(out, dim=1)
            _, predicted_idx = torch.max(probabilities, 1)
            predicted_label = class_labels[predicted_idx.item()]
            confidence = probabilities[0, predicted_idx.item()].item()

        return f"Predicted Class: {predicted_label} (Confidence: {confidence:.4f})"

    except Exception as e:
        return f"Error during classification: {e}"

demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="Image Classification with Trained Model",
    description="Upload an image to classify it based on the model it was trained on."
)

demo.launch()

ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [ ]:
## Create a Gradio interface for the image classification model
demo= gr.Interface(
    fn=lambda img: img. inputs= "image"
    outputs="text",
    title="Image Classification Demo", )
launch(show_error=True)
app.launch(show_error=True)

In [ ]:
## Define a custom theme for the Gradio app
gr.themes.builder()
class Seafoam (Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.background_color = "#008080"
        self.text_color = "#000000"
        self.button_color = "#800020"
        self.button_text_color = "#000000"
        self.input_background_color = "#FFFFFF"
        self.input_text_color = "#000000"
        self.output_background_color = "#FFFFFF"
        self.output_text_color = "#000000"
#   Create a Gradio Blocks app with the custom theme
    with gr.Blocks(theme=Seafoam(primary_hue="008080", secondary_hue="800020")) as demo:
        gr.Markdown("# Image Classification with ResNet50")
        gr.Markdown("Upload an image to classify it.")
        gr.Markdown ("ask a medical question")
        image_input = gr.Image(type="pil", label="Input Image")
        image_classify_button = gr.Button("Classify Image")
        output_text = gr.Textbox(label="Classification Result")
        textbox = gr.Textbox(label="Ask a medical question")
        text_classify_button = gr.Button("Ask")
        text_classify_button.click(predict_fn, inputs=image_input, outputs=output_text)
        text_classify_button.click(predict_fn, inputs=textbox, outputs=output_text)
# Set spacing and radius sizes for the Blocks app
with gr.Blocks(theme=gr.themes.Default(spacing_size="md", radius_size="md")) as demo:
# Set fonts for the Blocks app
with gr.Blocks(theme=gr.themes.Default(font=[gr.themes.GoogleFont("Helvetica"), "Arial", "sans-serif"])) as demo: